# IMPORT LIBRARIES

In [33]:
import pandas as pd
from tkinter import Tk, simpledialog, filedialog
from scipy import stats
from tabulate import tabulate


# EXTRACT DATA AND PERFORM TEST

In [34]:
def read_excel_file(file_path):
    """Reads an Excel file and returns a DataFrame."""
    try:
        df = pd.read_excel(file_path)  # Read Excel file into a temporary DataFrame
        df = df.dropna(axis=1, how='all')  # Drop columns that are entirely blank
        return df  # Return the DataFrame
    except Exception as e:
        print(f"Error reading Excel file: {e}")  # Print error message if file reading fails
        return None  # Return None if an error occurs

def perform_one_way_anova(df, group_cols):
    """Performs one-way ANOVA and displays additional statistics."""
    try:
        # Prepare data for ANOVA
        data = [df[col] for col in group_cols]

        # Perform one-way ANOVA
        f_stat, p_value = stats.f_oneway(*data)

        # Calculate degrees of freedom
        df_between = len(group_cols) - 1
        df_within = len(df) - len(group_cols)

        # Prepare data for table
        table_data = [
            ["F-statistic", round(f_stat, 3)],
            ["P-value", round(p_value, 3)],
            ["Degrees of freedom (between)", df_between],
            ["Degrees of freedom (within)", df_within]
        ]
        # Print table
        print("\nOne-way ANOVA results:")
        print(tabulate(table_data, headers=["Statistic", "Value"], tablefmt="grid"))

        # Interpret the results
        alpha = 0.05  # significance level
        if p_value < alpha:
            print("\nReject the null hypothesis: There is a significant difference between at least two groups.")
        else:
            print("\nFail to reject the null hypothesis: There is no significant difference between groups.")

    except Exception as e:
        print(f"Error performing one-way ANOVA: {e}")

def get_user_column_selection(df):
    """Gets user input for selecting columns using dialog boxes."""
    root = Tk()
    root.withdraw()  # Hide the main window

    # Get column names from the DataFrame
    columns = df.columns.tolist()

    while True:
        num_groups = simpledialog.askinteger("Input", "Enter the number of groups (must be 3 or more):")
        if num_groups is None:
            return None, None  # User canceled the input
        elif num_groups >= 3:
            break
        else:
            messagebox.showerror("Error", "Please enter 3 or more groups.")

    group_cols = []
    for i in range(num_groups):
        group_col = simpledialog.askstring("Input", f"Enter the column name for group {i + 1}:\nAvailable columns: {', '.join(columns)}")
        if group_col is None:
            return None, None  # User canceled the input
        group_cols.append(group_col)

    return group_cols

def main():
    # Open a file dialog to select Excel file
    root = Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
    if not file_path:
        print("No file selected.")
        return

    # Read Excel file into DataFrame
    df = read_excel_file(file_path)
    if df is None:
        print("Error reading Excel file.")
        return

    # Get user input for selecting columns
    group_cols = get_user_column_selection(df)
    if group_cols is None:
        print("Error: User canceled the input or entered invalid data.")
        return

    # Perform one-way ANOVA and display table
    perform_one_way_anova(df, group_cols)

if __name__ == "__main__":
    main()



One-way ANOVA results:
+------------------------------+---------+
| Statistic                    |   Value |
+==============================+=========+
| F-statistic                  |   2.657 |
+------------------------------+---------+
| P-value                      |   0.074 |
+------------------------------+---------+
| Degrees of freedom (between) |   2     |
+------------------------------+---------+
| Degrees of freedom (within)  |  46     |
+------------------------------+---------+

Fail to reject the null hypothesis: There is no significant difference between groups.
